In [3]:
import numpy as np
from keras.utils import to_categorical

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split



data1 = np.genfromtxt('xf.csv', delimiter=',')
data2 = np.genfromtxt('yf.csv', delimiter=',')


Using TensorFlow backend.
C:\Users\Thakker\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Thakker\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Thakker\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Thakker\anaconda3\lib\site-packages\tensorflow\python\framework\d

In [21]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(data1,data2, test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)


Training data shape :  (624, 784) (624,)


In [22]:
classes = np.unique(train_Y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Total number of outputs :  26
Output classes :  [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25.]


In [23]:
train_X = train_X.reshape(-1, 28,28, 1)
test_X = test_X.reshape(-1, 28,28, 1)
train_X.shape, test_X.shape

((624, 28, 28, 1), (156, 28, 28, 1))

In [24]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

In [25]:
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[1])
print('After conversion to one-hot:', train_Y_one_hot[1])

Original label: 7.0
After conversion to one-hot: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [26]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

In [27]:
train_X.shape,valid_X.shape,train_label.shape,valid_label.shape

((499, 28, 28, 1), (125, 28, 28, 1), (499, 26), (125, 26))

In [28]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [29]:
batch_size = 64
epochs = 50
num_classes = 26

In [30]:
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(28,28,1)))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Dropout(0.25))
fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.25))
fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.4))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))           
fashion_model.add(Dropout(0.3))
fashion_model.add(Dense(num_classes, activation='softmax'))

In [31]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [32]:
fashion_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)         

In [33]:
fashion_train_dropout = fashion_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

Train on 499 samples, validate on 125 samples
Epoch 1/50
499/499 [==============================] - 1s 3ms/step - loss: 3.2666 - accuracy: 0.0441 - val_loss: 3.2262 - val_accuracy: 0.0560
Epoch 2/50
499/499 [==============================] - 1s 2ms/step - loss: 3.1587 - accuracy: 0.1042 - val_loss: 3.1396 - val_accuracy: 0.1280
Epoch 3/50
499/499 [==============================] - 1s 2ms/step - loss: 2.9572 - accuracy: 0.1864 - val_loss: 2.7943 - val_accuracy: 0.2080
Epoch 4/50
499/499 [==============================] - 1s 2ms/step - loss: 2.5669 - accuracy: 0.2425 - val_loss: 2.2585 - val_accuracy: 0.4320
Epoch 5/50
499/499 [==============================] - 1s 2ms/step - loss: 2.1778 - accuracy: 0.3707 - val_loss: 2.0746 - val_accuracy: 0.3760
Epoch 6/50
499/499 [==============================] - 1s 2ms/step - loss: 1.8118 - accuracy: 0.4349 - val_loss: 1.6464 - val_accuracy: 0.5120
Epoch 7/50
499/499 [==============================] - 1s 2ms/step - loss: 1.5737 - accuracy: 0.5070 - 

In [34]:
fashion_model.save("fashion_model_dropout.h5py")

In [35]:

test_eval = fashion_model.evaluate(test_X, test_Y_one_hot, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

Test loss: 0.4158020187646915
Test accuracy: 0.8974359035491943


In [51]:
from matplotlib.image import imread


img = imread('g.png')
img = img[:,:,0]

a = np.array(img)

b = a.ravel()



b=b.reshape(1,28,28,1)
predicted_classes=fashion_model.predict(b)
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

print("Character found is "+chr(int(predicted_classes+65)))

Character found is I
